In [1]:
import time
import random
import numpy as np
import importlib

import sys
sys.path.append("../../")

In [2]:
from qiskit import QuantumProgram, QuantumCircuit
import Qconfig

In [3]:
import tools.Experiment_tools as exptool
importlib.reload(exptool)

<module 'tools.Experiment_tools' from '/home/christophe/Work/Programming/IBMQE/qiskit-sdk-py/examples/experimenting_with_fault_tolerance/tools/Experiment_tools.py'>

In [4]:
qprogram = QuantumProgram()
qprogram.set_api(Qconfig.APItoken, Qconfig.config["url"])

In [5]:
qprogram.available_backends()

['ibmqx4',
 'ibmqx2',
 'ibmqx5',
 'ibmqx_qasm_simulator',
 'local_qasm_simulator',
 'local_unitary_simulator']

In [17]:
qprogram.get_backend_status("ibmqx4")

{'available': True, 'backend': 'ibmqx4', 'busy': False, 'pending_jobs': 12}

In [7]:
config = qprogram.get_backend_configuration("ibmqx4",list_format=True)
config

{'basis_gates': 'SU2+CNOT',
 'chip_name': 'Raven',
 'coupling_map': [[1, 0], [2, 0], [2, 1], [2, 4], [3, 2], [3, 4]],
 'description': '5 qubit transmon bowtie chip 3',
 'n_qubits': 5,
 'name': 'ibmqx4',
 'online_date': '2017-09-18T11:00:00.000Z',
 'simulator': False,
 'url': 'https://ibm.biz/qiskit-ibmqx4',
 'version': '1'}

In [8]:
n = config['n_qubits']
q = qprogram.create_quantum_register("q",n)
c = qprogram.create_classical_register("c",n)

In [9]:
circuit_names = exptool.all_circuits(qprogram,config['coupling_map'])
print(len(circuit_names))
print(len(qprogram.get_circuit_names()))

176
245


In [10]:
print(qprogram.get_circuit("bMX2-CZ-HHS-X1|00>[3, 4]").qasm())

OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
x q[3];
h q[3];
h q[4];
h q[4];
cx q[3],q[4];
h q[4];
x q[3];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];



In [11]:
rand_perm_names = np.random.permutation(circuit_names)
batches = [list(rand_perm_names[50*j:50*(j+1)]) for j in range(0,1+int(len(circuit_names)/50))]

In [12]:
compiled_qobj_list = [qprogram.compile(batch, 
                                       backend='ibmqx4', 
                                       config=None,
                                       silent=False, 
                                       basis_gates=None, 
                                       coupling_map=None, 
                                       initial_layout=None, 
                                       shots=8192, 
                                       max_credits=5, 
                                       seed=None) for batch in batches]

In [18]:
qprogram.run_batch_async(compiled_qobj_list, wait=30, timeout=1800, silent=True, callback=exptool.post_treatment_list)

In [31]:
print(run.get_names())
print(run.get_ran_qasm('bM|00>[3, 4]'))
print(run.get_ran_qasm('bM|00>[1, 0]'))
run.get_data('bM|00>[3, 4]')

['bM|00>[3, 4]', 'bM|00>[1, 0]']
OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
measure q[4] -> c[4];
measure q[3] -> c[3];
measure q[2] -> c[2];
measure q[1] -> c[1];
measure q[0] -> c[0];

OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[5];
measure q[4] -> c[4];
measure q[3] -> c[3];
measure q[2] -> c[2];
measure q[1] -> c[1];
measure q[0] -> c[0];



QISKitError: 'Time Out'

In [36]:
res = qprogram.jobs_results[0]
res.get_status()

'ERROR'

In [15]:
results = qprogram.jobs_results
result = results[0]
len(results)

1

In [16]:
result.get_status()

'ERROR'

In [47]:
results = qprogram.execute(["bM|00>[3, 4]","bMX1|00>[3, 4]"], backend='ibmqx4', shots=1000, max_credits=10, wait=10, timeout=760)

In [54]:
if results.get_status()=='COMPLETED':
    print(results.get_counts("bM|00>[3, 4]"))
    print(results.get_data("bMX1|00>[3, 4]"))
else:
    print(results.get_status())
results.get_names()

{'10000': 10, '00000': 915, '00001': 6, '00010': 13, '00100': 16, '01000': 39, '01100': 1}
{'time': 48.44315791130066, 'counts': {'10000': 1, '11000': 6, '00000': 118, '00001': 1, '00010': 1, '00100': 3, '01000': 829, '01001': 9, '01010': 8, '01100': 24}, 'date': '2017-10-03T12:28:52.339Z'}


['bM|00>[3, 4]', 'bMX1|00>[3, 4]']

In [91]:
calib = qprogram.get_backend_calibration("ibmqx4")
print('Gate error :\n------------')
print([item['gateError']['value'] for item in calib['qubits']])
print('\nReadout error :\n---------------')
print([item['readoutError']['value'] for item in calib['qubits']])
print('Multi qubit gate error :\n------------')
print([item['gateError']['value'] for item in calib['multi_qubit_gates']])
calib

Gate error :
------------
[0.0006009179709377532, 0.0006867731322012238, 0.001631340796924452, 0.0014595792651160289, 0.0017172252255832077]

Readout error :
---------------
[0.033, 0.029, 0.026, 0.134, 0.036]
Multi qubit gate error :
------------
[0.020000606743482918, 0.022706065613666004, 0.024738466135747345, 0.017300151987441892, 0.02236760832218937, 0.025213102663840553]


{'backend': 'ibmqx4',
 'last_update_date': '2017-10-02T11:02:06.000Z',
 'multi_qubit_gates': [{'gateError': {'date': '2017-10-02T11:02:06Z',
    'value': 0.020000606743482918},
   'name': 'CX1_0',
   'qubits': [1, 0],
   'type': 'CX'},
  {'gateError': {'date': '2017-10-02T11:02:06Z',
    'value': 0.022706065613666004},
   'name': 'CX2_0',
   'qubits': [2, 0],
   'type': 'CX'},
  {'gateError': {'date': '2017-10-02T11:02:06Z',
    'value': 0.024738466135747345},
   'name': 'CX2_1',
   'qubits': [2, 1],
   'type': 'CX'},
  {'gateError': {'date': '2017-10-02T11:02:06Z',
    'value': 0.017300151987441892},
   'name': 'CX2_4',
   'qubits': [2, 4],
   'type': 'CX'},
  {'gateError': {'date': '2017-10-02T11:02:06Z', 'value': 0.02236760832218937},
   'name': 'CX3_2',
   'qubits': [3, 2],
   'type': 'CX'},
  {'gateError': {'date': '2017-10-02T11:02:06Z',
    'value': 0.025213102663840553},
   'name': 'CX3_4',
   'qubits': [3, 4],
   'type': 'CX'}],
 'qubits': [{'gateError': {'date': '2017-10-02T1

In [ ]:
param = qprogram.get_backend_parameters("ibmqx4")
print('T1:\n---')
print([(item['T1']['value'],item['T1']['unit']) for item in param['qubits']])
print('\nT2:\n---')
print([(item['T2']['value'],item['T2']['unit']) for item in param['qubits']])
param

In [ ]:
# Find a reasonable formula to automatically choose the best two qubits.

In [ ]:
possible_pairs = config['coupling_map']
print(possible_pairs)
cp = 3 #the chosen pair
print(possible_pairs[cp])

In [ ]:
n = config['n_qubits']
q = qprogram.create_quantum_register("q",n)
c = qprogram.create_classical_register("c",n)

In [ ]:
qc_FT = qprogram.create_circuit("prep_FT",[q],[c])
qc_FT.h(q[2])
qc_FT.cx(q[2],q[0])
qc_FT.cx(q[2],q[1])
qc_FT.h(q[2])
qc_FT.h(q[3])
qc_FT.cx(q[3],q[2])
qc_FT.h(q[2])
qc_FT.h(q[3])
qc_FT.cx(q[2],q[4])
qc_FT.cx(q[2],q[0])
qc_FT.measure(q[0],c[0])
qc_FT.measure(q[1],c[1])
qc_FT.measure(q[2],c[2])
qc_FT.measure(q[3],c[3])
qc_FT.measure(q[4],c[4])

In [ ]:
def swap_circuit(qr,pair,qp):
    qcircuitswap = qp.create_circuit("SWAP"+str(pair),[qr],[])
    qcircuitswap.cx(qr[pair[0]],qr[pair[1]])
    qcircuitswap.h(qr[pair[0]])
    qcircuitswap.h(qr[pair[1]])
    qcircuitswap.cx(qr[pair[0]],qr[pair[1]])
    qcircuitswap.h(qr[pair[0]])
    qcircuitswap.h(qr[pair[1]])
    qcircuitswap.cx(qr[pair[0]],qr[pair[1]])
    return qcircuitswap

In [ ]:
qc_NFT = qprogram.create_circuit("prep_NFT",[q],[c])
qc_NFT.h(q[3])
qc_NFT.cx(q[3],q[4])
qc_NFT.cx(q[3],q[2])
qc_NFT.cx(q[2],q[1])
qc_NFT.measure(q[0],c[0])
qc_NFT.measure(q[1],c[1])
qc_NFT.measure(q[2],c[2])
qc_NFT.measure(q[3],c[3])
qc_NFT.measure(q[4],c[4])

In [ ]:
qc_NFTold = qprogram.create_circuit("prep_NFTold",[q],[c])
qc_NFTold.h(q[2])
qc_NFTold.h(q[3])
qc_NFTold.cx(q[3],q[4])
qc_NFTold.h(q[4])
qc_NFTold.cx(q[2],q[4])
qc_NFTold.h(q[4])
qc_NFTold.extend(swap_circuit(q,[2,1],qprogram))
qc_NFTold.cx(q[3],q[2])
qc_NFTold.cx(q[2],q[0])
qc_NFTold.h(q[0])
qc_NFTold.cx(q[1],q[0])
qc_NFTold.h(q[0])
qc_NFTold.h(q[1])
qc_NFTold.measure(q[0],c[0])
qc_NFTold.measure(q[1],c[1])
qc_NFTold.measure(q[2],c[2])
qc_NFTold.measure(q[3],c[3])
qc_NFTold.measure(q[4],c[4])

In [ ]:
qc_FTv2 = qprogram.create_circuit("prep_FTv2",[q],[c])
qc_FTv2.h(q[3])
qc_FTv2.cx(q[3],q[2])
qc_FTv2.h(q[2])
qc_FTv2.h(q[3])
qc_FTv2.cx(q[2],q[1])
qc_FTv2.cx(q[3],q[4])
qc_FTv2.h(q[4])
qc_FTv2.extend(swap_circuit(q,[2,4],qprogram))
qc_FTv2.cx(q[2],q[0])
qc_FTv2.cx(q[1],q[0])
qc_FTv2.h(q[4])
qc_FTv2.measure(q[0],c[0])
qc_FTv2.measure(q[1],c[1])
qc_FTv2.measure(q[2],c[2])
qc_FTv2.measure(q[3],c[3])
qc_FTv2.measure(q[4],c[4])

In [ ]:
def prep_bare_circuit(pair,qprogram, q, c):
    qc_ppBare = qprogram.create_circuit("prep_Bare - "+str(pair),[q],[c])
    qc_ppBare.measure(q[pair[0]],c[pair[0]])
    qc_ppBare.measure(q[pair[1]],c[pair[1]])
    return qc_ppBare
qc_ppBare = prep_bare_circuit(possible_pairs[cp],qprogram,q,c)

In [ ]:
result = qprogram.execute(["prep_FT","prep_FTv2","prep_NFTold","prep_NFT","prep_Bare - "+str(possible_pairs[cp])], "ibmqx4", shots=8192, max_credits=5, wait=10, timeout=2400)

In [ ]:
print(result.get_counts("prep_FT"))
print(result.get_counts("prep_FTv2"))
print(result.get_counts("prep_NFT"))
print(result.get_counts("prep_NFTold"))
print(result.get_counts("prep_Bare - "+str(possible_pairs[cp])))

In [ ]:
from qiskit.tools.visualization import plot_state, plot_histogram

In [ ]:
plot_histogram(result.get_counts("prep_FT"))
plot_histogram(result.get_counts("prep_FTv2"))
plot_histogram(result.get_counts("prep_NFT"))
plot_histogram(result.get_counts("prep_NFTold"))
plot_histogram(result.get_counts("prep_Bare - "+str(possible_pairs[cp])))

In [ ]:
result.get_ran_qasm("prep_Bare")

In [ ]:
def encoded_X1_circuit(mapping, qp):
    qrs = [qp.get_quantum_register(qrn) for qrn in qp.get_quantum_register_names()]
    crs = [qp.get_classical_register(crn) for crn in qp.get_classical_register_names()]
    qcircuit_encoded_X1 = qp.create_circuit("encoded_X1",qrs,crs)
    qcircuit_encoded_X1.x(qrs[0][mapping[0]])
    qcircuit_encoded_X1.x(qrs[0][mapping[1]])
    return qcircuit_encoded_X1
def encoded_X2_circuit(qr, mapping, qp):
    qcircuit_encoded_X2 = qp.create_circuit("encoded_X2",[qr],[])
    qcircuit_encoded_X2.x(qr[mapping[0]])
    qcircuit_encoded_X2.x(qr[mapping[2]])
    return qcircuit_encoded_X2
def encoded_Z1_circuit(qr, mapping, qp):
    qcircuit_encoded_Z1 = qp.create_circuit("encoded_Z1",[qr],[])
    qcircuit_encoded_Z1.z(qr[mapping[1]])
    qcircuit_encoded_Z1.z(qr[mapping[3]])
    return qcircuit_encoded_Z1
def encoded_Z2_circuit(qr, mapping, qp):
    qcircuit_encoded_Z2 = qp.create_circuit("encoded_Z2",[qr],[])
    qcircuit_encoded_Z2.z(qr[mapping[2]])
    qcircuit_encoded_Z2.z(qr[mapping[3]])
    return qcircuit_encoded_Z2
def encoded_CZ_circuit(qr, mapping, qp):
    qcircuit_encoded_CZ = qp.create_circuit("encoded_CZ",[qr],[])
    qcircuit_encoded_CZ.s(qr[mapping[0]])
    qcircuit_encoded_CZ.s(qr[mapping[1]])
    qcircuit_encoded_CZ.s(qr[mapping[2]])
    qcircuit_encoded_CZ.s(qr[mapping[3]])
    return qcircuit_encoded_CZ
def encoded_HHS_circuit(qr, mapping, qp):
    qcircuit_encoded_HHS = qp.create_circuit("encoded_HHS",[qr],[])
    qcircuit_encoded_HHS.h(qr[mapping[0]])
    qcircuit_encoded_HHS.h(qr[mapping[1]])
    qcircuit_encoded_HHS.h(qr[mapping[2]])
    qcircuit_encoded_HHS.h(qr[mapping[3]])
    return qcircuit_encoded_HHS

In [ ]:
encoded_X1_circuit([0,1],qprogram).qasm()

In [ ]:
#print(qprogram.compile(["prep_FT"],backend="ibmqx4")['circuits'][0]['compiled_circuit_qasm'])

In [ ]:
import csv

with open('mycsvfile5.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, result.get_counts("prep_FT").keys())
    w.writeheader()
    w.writerows([result.get_counts("prep_FT"),result.get_counts("prep_FTv2"),result.get_counts("prep_NFT"),result.get_counts("prep_NFTold"),result.get_counts("prep_Bare - "+str(possible_pairs[cp]))])

In [ ]:
print(result.get_ran_qasm("prep_FT") + '\n')
print(result.get_ran_qasm("prep_FTv2") + '\n')
print(result.get_ran_qasm("prep_NFT") + '\n')
print(result.get_ran_qasm("prep_NFTold") + '\n')
print(result.get_ran_qasm("prep_Bare - "+str(possible_pairs[cp])) + '\n')

In [ ]:
def convert_rate(rT, rG):
    if rT['unit']=='ns':
        mT = 10**(-9)
    elif rT['unit']=='µs':
        mT = 10**(-6)
    else:
        mT = 1
    if rG['unit']=='ns':
        mG = 10**(-9)
    elif rG['unit']=='µs':
        mG = 10**(-6)
    else:
        mG = 1
    return 1-np.exp(-rG['value']*mG/(rT['value']*mT))
        

def rank_qubit_pairs(qp,backend):
    config = qp.get_backend_configuration(backend,list_format=True)
    calib = qp.get_backend_calibration(backend)
    param = qp.get_backend_parameters(backend)
    n = config['n_qubits']
    pairs_numbers = dict([(tuple(c),[]) for c in config['coupling_map']])
    for p,s in pairs_numbers.items():
        s.append(calib['qubits'][p[0]]['gateError']['value'])
        s.append(calib['qubits'][p[0]]['readoutError']['value'])
        s.append(calib['qubits'][p[1]]['gateError']['value'])
        s.append(calib['qubits'][p[1]]['readoutError']['value'])
        for r in calib['multi_qubit_gates']:
            if r['qubits']==list(p):
                s.append(r['gateError']['value'])
        s.append(convert_rate(param['qubits'][p[0]]['T1'],param['qubits'][p[0]]['gateTime']))
        s.append(convert_rate(param['qubits'][p[0]]['T2'],param['qubits'][p[0]]['gateTime']))
        s.append(convert_rate(param['qubits'][p[1]]['T1'],param['qubits'][p[1]]['gateTime']))
        s.append(convert_rate(param['qubits'][p[1]]['T2'],param['qubits'][p[1]]['gateTime']))
    for p,s in pairs_numbers.items():
        pairs_numbers[p] = sorted(s,reverse=True)
    return (sorted(pairs_numbers,key=pairs_numbers.__getitem__),pairs_numbers)

        
        
    
    
    
    


    

In [ ]:
(sorted_pairs,pairs_dict) = rank_qubit_pairs(qprogram,"ibmqx4")

In [ ]:
sorted_pairs

In [ ]:
bare_programs_list = []
bare_program_names_list = []
for p in sorted_pairs:
    bare_programs_list.append(prep_bare_circuit(p,qprogram,q,c))
    bare_program_names_list.append("prep_Bare - "+str(p))


In [ ]:
bare_program_names_list

In [ ]:
result_bare = qprogram.execute(bare_program_names_list, "ibmqx4", shots=8192, max_credits=5, wait=10, timeout=2400)

In [ ]:
with open('mycsvfile_bare2.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, [('{0:0'+str(n)+'b}').format(j) for j in range(0,2**n)])
    w.writeheader()
    w.writerows([result_bare.get_counts(name) for name in bare_program_names_list])

In [33]:
[('{0:0'+str(n)+'b}').format(j) for j in range(0,2**n)]

['00000',
 '00001',
 '00010',
 '00011',
 '00100',
 '00101',
 '00110',
 '00111',
 '01000',
 '01001',
 '01010',
 '01011',
 '01100',
 '01101',
 '01110',
 '01111',
 '10000',
 '10001',
 '10010',
 '10011',
 '10100',
 '10101',
 '10110',
 '10111',
 '11000',
 '11001',
 '11010',
 '11011',
 '11100',
 '11101',
 '11110',
 '11111']

In [118]:
int(len(circuit_names)/50)

3

In [14]:
l = [1,2,3,4,5,6,7]

In [15]:
l[0:1]

[1]

In [97]:
a = '3ndfo4O'
a[len(a)-2:len(a)]

'4O'